In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

In [ ]:
!pip install accelerate -U

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pandas as pd
from datasets import Dataset

# load data
df = pd.read_csv("/content/drive/MyDrive/Notebooks/dataset/processed_data_utf8.csv")
texts = df['Content'].tolist() 

# initialize tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# tokenize data
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = Dataset.from_dict({"text": texts})
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# initialize data_collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

# initialize model
model = XLMRobertaForMaskedLM.from_pretrained('xlm-roberta-base')

# initialize training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

# initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets,
)

# train model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/77492 [00:00<?, ? examples/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 

Step,Training Loss
500,3.179600
1000,2.360600
1500,2.172200
2000,1.950900
2500,1.863900
3000,1.754800
3500,1.708100
4000,1.607800
4500,1.618300
5000,1.536600


TrainOutput(global_step=19373, training_loss=1.4129542454785655, metrics={'train_runtime': 14877.6593, 'train_samples_per_second': 5.209, 'train_steps_per_second': 1.302, 'total_flos': 2.044851575795712e+16, 'train_loss': 1.4129542454785655, 'epoch': 1.0})

In [ ]:
# save model
trainer.save_model("/content/drive/MyDrive/Notebooks/Models/xlm_trained_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Notebooks/Models/xlm_trained_model")


NameError: name 'trainer' is not defined

In [4]:
from transformers import XLMRobertaForMaskedLM, XLMRobertaTokenizer

model_path = "/content/drive/MyDrive/Notebooks/Models/xlm_trained_model"

# Reload model and tokenizer
model = XLMRobertaForMaskedLM.from_pretrained(model_path)
